In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os.path
import random
import shutil
import datetime
import numpy as np
import DataProcessing as DP
import pandas as pd
import multiprocessing
from functools import reduce
import ML as ML
from absl import app
from absl import flags
import numpy as np
import six
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPool1D, BatchNormalization, Flatten, Dense,Dropout, Dropout, Input, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers, regularizers
from sklearn.utils import shuffle
import tensorflow_addons as tfa


from  DataProcessing import realized_volatility, trainValidSplitNp, CutAndLinear, bookDataPreprocess, combineForTraining, npToDataset
from DataProcessing import RawDataInitialization, NpRepeatByCondition, NpSplitAndNormalized

from tensorboard.plugins.hparams import api as hp

from ML import *
import ML as ML
from  DataProcessing import *
import DataProcessing as DP
import pandas as pd
import numpy as np
import os
import glob
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPool1D, BatchNormalization, Flatten, Dense,Dropout, Dropout, Input, Concatenate, Conv2D, MaxPooling2D
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import time
import math 
from tensorboard.plugins.hparams import api as hp

import warnings
warnings.simplefilter('ignore')

In [12]:
linearResults = pd.read_csv('LinearRegressionResults.csv')

train_book, train_file, train_trade = DP.RawDataInitialization(range(0,130))
list_of_stocks = train_book.keys()
         
data_frames = [pd.read_csv('globalData/'+stock_id+'.csv') for stock_id in list_of_stocks]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['time_id', 'split_60s'],
                                             how='outer'), data_frames)
df_merged_reindexed = df_merged.set_index(['time_id', 'split_60s']).fillna(0)

df = pd.read_csv('4_inputs_results.csv', index_col=0)

In [13]:
list_of_stocks_skip = df.index
mse_4inputs = {}
# for stock_id in list(train_book.keys()):
stock_id='stock_id=0'
# if stock_id  in list_of_stocks_skip:
#     continue
print("Now starts {stock_id}".format(stock_id = stock_id))
# stock_id = 'stock_id=52'
stock_id_as_int = stock_id.split('=')[1]
data, labels = DP.RawDataToBookAndTradeNp(stock_id_as_int)
train_data, train_labels, valid_data, valid_labels = DP.trainValidSplitNp(data, labels, 0.85)
mean = train_labels.mean()
std = train_labels.std()
train_labelsNormalized, valid_labelsNormalized = (train_labels-mean)/std, (valid_labels-mean)/std   


np_test = np.transpose(np.array(df_merged_reindexed.to_xarray().to_array()), [1,2,0]).reshape(3830, 6, 112,2)
cut = train_data.shape[0]
globalTrain = np_test[:cut]
globalValid = np_test[cut:]
if (data.shape[0] != 3830):
    with open('MissingTimeId.txt', 'w') as f:
        f.writelines("{stock} has missing time_id".format(stock = stock_id))
#     continue

interval = 120
df2 = DP.RawDataToPd(train_book[stock_id], train_file, stock_id)
df2['split_60s'] = df2['seconds_in_bucket']//interval
df3 =  pd.DataFrame(df2.groupby(['stock_id','time_id','split_60s'])['log_return'].apply(realized_volatility))

interval = 30
# df4 = DP.RawDataToPd(train_book[stock_id], train_file, stock_id)
df4 = DP.RawDataToBookAndTradePd(60)
df4['split_{interval}s'.format(interval = interval)] = df4['seconds_in_bucket']//interval
df5 =  pd.DataFrame(df4.groupby(['stock_id',
                                 'time_id',
                                 'split_{interval}s'.format(interval = interval)])[['log_return', 'log_return_trade']].apply(realized_volatility))

dataFromIntervals = np.array(df3.unstack())
dataFromSmallIntervals =np.array(df5.unstack())
dataFromSmallIntervals = np.transpose(dataFromSmallIntervals.reshape(3830, 2, 600//interval),[0,2,1]) #2 = features = length of ['log_return', 'log_return_trade']

trainFromInterval = dataFromIntervals[:cut]
validFromInterval = dataFromIntervals[cut:]

trainFromSmallInterval = dataFromSmallIntervals[:cut]
validFromSmallInterval = dataFromSmallIntervals[cut:]

trainDataset = tf.data.Dataset.from_tensor_slices(({"input_600s": train_data,
                                                    "input_global": globalTrain,
                                                    'input_intervals':trainFromInterval,
                                                    'input_smallIntervals': trainFromSmallInterval }, 
                                                   train_labelsNormalized)).batch(ML.BATCH_SIZE, drop_remainder=True).repeat(3)

validDataset = tf.data.Dataset.from_tensor_slices(({"input_600s": valid_data, 
                                                    "input_global": globalValid, 
                                                    'input_intervals': validFromInterval,
                                                    'input_smallIntervals': validFromSmallInterval}, 
                                                   valid_labelsNormalized)).batch(ML.BATCH_SIZE, drop_remainder=True)

Now starts stock_id=0


# Transformer using Gelu

In [14]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from random import choices
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GroupKFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation

#simply think of it as a matrix multiplication function. In reality depth = number of features, seq_len_q = seq_len_k = seq_len_v
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b = True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:

        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
    """
    for a input of (batch_size, seq_length, features), with parameters = (heads, depth). Let d_model = heads * depth
    first use three (features , (depth * heads)) to map it into three (batch, seq_length, d_model)
    starting from the second layer, use (d_model, d_model) to do the same thing.
    """

    def __init__(self, d_model, num_heads):

        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm = [0, 2, 1, 3])

    def call(self, v, k, q, mask):

        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

def gelu(x):
    """Gaussian Error Linear Unit.
    This is a smoother version of the RELU.
    Original paper: https://arxiv.org/abs/1606.08415
    refer : https://github.com/google-research/bert/blob/bee6030e31e42a9394ac567da170a89a98d2062f/modeling.py#L264
    Args:
        x: float Tensor to perform activation.
    Returns:
        `x` with the GELU activation applied.
    """
    cdf = 0.5 * (1.0 + tf.tanh(
        (np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

def point_wise_feed_forward_network(d_model, dff):

    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation = gelu),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, dff, rate = 0.1):

        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training = training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class TransformerEncoder(tf.keras.layers.Layer):

    def __init__(self, num_layers, d_model, num_heads, dff, rate = 0.1):

        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate

        self.embedding = tf.keras.layers.Dense(self.d_model)

        self.enc_layers = [EncoderLayer(self.d_model, self.num_heads, self.dff, self.rate)
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(self.rate)

    def get_config(self):
        """
        layer.get_config() can return config of this layer, and then a new layer can be 
        initialized using Layer.from_config().
        """

        config = super().get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'dropout': self.dropout,
        })
        return config

    def call(self, x, training, mask = None):

        seq_len = tf.shape(x)[1]

        x = self.embedding(x)

        x = self.dropout(x, training = training)

        for i in range(self.num_layers):

            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

## An Example of Transformer

In [15]:
def single_model(input_shape, num_columns, num_labels, weights = None):
    d_model = 5
    num_heads = 10
    inp = tf.keras.layers.Input(shape=input_shape)
    x = TransformerEncoder(2, d_model*num_heads, num_heads, 64)(inp)
    x = tf.keras.layers.Flatten()(x)
    out = tf.keras.layers.Dense(num_labels, activation = 'linear')(x)
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = 'adam', 
                  loss = tf.keras.losses.MeanSquaredError(), metrics = [tf.keras.metrics.MeanSquaredError()])   
    return model

In [24]:
left_inputs = Input(shape=(600, 2), name='input_600s')
x = left_inputs
filters = 32
dropout_rate = 0.35
weight_decay = 0.012
x = Normalization()(x)
for i in range(2):
    x = Conv1D(filters = filters,
               kernel_size= 3,
               padding = 'same',
               activation = 'relu',
               kernel_regularizer=regularizers.l1(weight_decay)
               )(x)
    x = Dropout(dropout_rate)(x)
    x = MaxPool1D()(x)
    filters *= 2
x = Flatten()(x)
x = Dense(units = 32, 
          activation='relu'
          )(x)
x = Dense(1)(x)
# =============================================================================
# second input: 10s per interval data
# =============================================================================
inputs_smallIntervals = Input(shape = (20,2), name = 'input_smallIntervals')
x2 = inputs_smallIntervals
# x2 = Normalization()(x2)
x2 = Conv1D(filters = 16,
            kernel_size=3,
            padding='same',
            activation='relu',
            kernel_regularizer=regularizers.l1(weight_decay))(x2)
x2 = MaxPool1D(2)(x2)
x2 = SimpleRNN(32, 
          return_sequences=False, 
          return_state=False)(x2)
x2 = Dropout(dropout_rate)(x2)
x2 = Flatten()(x2)
x2 = Dense(32, activation='relu')(x2)
x2 = Dense(1)(x2)

"""
transformer layer
"""
d_model = 3
num_heads = 10
x3 = TransformerEncoder(2, d_model*num_heads, num_heads, 64)(inputs_smallIntervals)
x3 = tf.keras.layers.Flatten()(x3)
x3 = Dense(32, activation='relu')(x3)
# out = tf.keras.layers.Dense(num_labels, activation = 'linear')(x3)



# =============================================================================
# third input: global data
# =============================================================================
right_inputs = Input(shape = (6, 112, 2), name='input_global')
y = right_inputs
y = Normalization()(y)
# filters = 64
for i in range(1):
    y = Conv2D(filters = filters,
               kernel_size=3,
               padding='same',
               activation='relu',
               kernel_regularizer=regularizers.l1(weight_decay)
               )(y)
    y = Dropout(dropout_rate)(y)
    y = MaxPooling2D()(y)
    filters *= 2
y = Flatten()(y)
y = Dense(units = 48,
          activation='relu',
          )(y)
y = Dense(1)(y)
# =============================================================================
# 4th input: 120s per interval data
# =============================================================================
inputs_intervals = Input(shape = (5,), name = 'input_intervals')
z = inputs_intervals
z = Normalization()(z)
z = Flatten()(z)
z = Dense(32,activation='relu')(z)
z = Dense(1)(z)

w = Concatenate()([x, x2,x3, y , z])
# w = BatchNormalization()(w)
w = Flatten()(w)
# w = Dense(32,
#           activation='relu',
#           kernel_regularizer=regularizers.l1(weight_decay))(w)
# w = Concatenate()([w,z])
# w = Dense(8, activation='relu')(w)
outputs = Dense(1)(w)

model = Model([left_inputs, right_inputs, inputs_intervals, inputs_smallIntervals], outputs)
model.compile(loss =  'mse',
              optimizer= 'adam',
              metrics = [tf.keras.metrics.MeanSquaredError()])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_600s (InputLayer)         [(None, 600, 2)]     0           []                               
__________________________________________________________________________________________________
normalization_13 (Normalizatio  (None, 600, 2)       5           ['input_600s[0][0]']             
n)                                                                                                
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 600, 32)      224         ['normalization_13[0][0]']       
__________________________________________________________________________________________________
dropout_39 (Dropout)            (None, 600, 32)      0           ['conv1d_15[0][0]']        

In [25]:
check_point, early_stopping, learning_rate_reduction = ML.get_callbacks('test', 
                                                                        monitor='val_mean_squared_error', mode='min')
model.fit(trainDataset, validation_data=validDataset, epochs=2000, callbacks=[early_stopping])

Epoch 1/2000
18/18 [==============================] - 7s 69ms/step - loss: 7.3981 - mean_squared_error: 1.4556 - val_loss: 5.9349 - val_mean_squared_error: 0.8740
Epoch 2/2000
18/18 [==============================] - 1s 46ms/step - loss: 5.4433 - mean_squared_error: 1.0801 - val_loss: 4.4718 - val_mean_squared_error: 0.8404
Epoch 3/2000
18/18 [==============================] - 1s 46ms/step - loss: 4.1223 - mean_squared_error: 1.0591 - val_loss: 3.3045 - val_mean_squared_error: 0.8253
Epoch 4/2000
18/18 [==============================] - 1s 46ms/step - loss: 3.1011 - mean_squared_error: 1.0526 - val_loss: 2.4422 - val_mean_squared_error: 0.8236
Epoch 5/2000
18/18 [==============================] - 1s 46ms/step - loss: 2.3546 - mean_squared_error: 1.0517 - val_loss: 1.8020 - val_mean_squared_error: 0.8172
Epoch 6/2000
18/18 [==============================] - 1s 47ms/step - loss: 1.7898 - mean_squared_error: 1.0340 - val_loss: 1.3441 - val_mean_squared_error: 0.8091
Epoch 7/2000
18/18 [==

Epoch 51/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.3111 - mean_squared_error: 0.2613 - val_loss: 0.3458 - val_mean_squared_error: 0.2961
Epoch 52/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.3102 - mean_squared_error: 0.2608 - val_loss: 0.3443 - val_mean_squared_error: 0.2952
Epoch 53/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.3075 - mean_squared_error: 0.2585 - val_loss: 0.3404 - val_mean_squared_error: 0.2914
Epoch 54/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.3059 - mean_squared_error: 0.2571 - val_loss: 0.3414 - val_mean_squared_error: 0.2927
Epoch 55/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.3035 - mean_squared_error: 0.2550 - val_loss: 0.3336 - val_mean_squared_error: 0.2852
Epoch 56/2000
18/18 [==============================] - 1s 48ms/step - loss: 0.3017 - mean_squared_error: 0.2535 - val_loss: 0.3300 - val_mean_squared_error: 0.2818
Epoch 57/2000
18

Epoch 101/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.2446 - mean_squared_error: 0.2021 - val_loss: 0.2442 - val_mean_squared_error: 0.2017
Epoch 102/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2424 - mean_squared_error: 0.2000 - val_loss: 0.2422 - val_mean_squared_error: 0.1999
Epoch 103/2000
18/18 [==============================] - 1s 48ms/step - loss: 0.2419 - mean_squared_error: 0.1996 - val_loss: 0.2461 - val_mean_squared_error: 0.2037
Epoch 104/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2393 - mean_squared_error: 0.1971 - val_loss: 0.2425 - val_mean_squared_error: 0.2002
Epoch 105/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.2411 - mean_squared_error: 0.1989 - val_loss: 0.2462 - val_mean_squared_error: 0.2039
Epoch 106/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2386 - mean_squared_error: 0.1965 - val_loss: 0.2364 - val_mean_squared_error: 0.1943
Epoch 107/

Epoch 151/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2196 - mean_squared_error: 0.1828 - val_loss: 0.2142 - val_mean_squared_error: 0.1773
Epoch 152/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.2195 - mean_squared_error: 0.1828 - val_loss: 0.2130 - val_mean_squared_error: 0.1764
Epoch 153/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.2193 - mean_squared_error: 0.1827 - val_loss: 0.2130 - val_mean_squared_error: 0.1764
Epoch 154/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.2193 - mean_squared_error: 0.1829 - val_loss: 0.2131 - val_mean_squared_error: 0.1765
Epoch 155/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.2172 - mean_squared_error: 0.1808 - val_loss: 0.2136 - val_mean_squared_error: 0.1772
Epoch 156/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.2202 - mean_squared_error: 0.1840 - val_loss: 0.2122 - val_mean_squared_error: 0.1759
Epoch 157/

Epoch 201/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.2124 - mean_squared_error: 0.1797 - val_loss: 0.2030 - val_mean_squared_error: 0.1704
Epoch 202/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.2134 - mean_squared_error: 0.1809 - val_loss: 0.2036 - val_mean_squared_error: 0.1710
Epoch 203/2000
18/18 [==============================] - 1s 48ms/step - loss: 0.2127 - mean_squared_error: 0.1802 - val_loss: 0.2037 - val_mean_squared_error: 0.1711
Epoch 204/2000
18/18 [==============================] - 1s 48ms/step - loss: 0.2112 - mean_squared_error: 0.1789 - val_loss: 0.2039 - val_mean_squared_error: 0.1714
Epoch 205/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.2128 - mean_squared_error: 0.1804 - val_loss: 0.2027 - val_mean_squared_error: 0.1703
Epoch 206/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.2121 - mean_squared_error: 0.1798 - val_loss: 0.2013 - val_mean_squared_error: 0.1690
Epoch 207/

Epoch 251/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2068 - mean_squared_error: 0.1771 - val_loss: 0.1995 - val_mean_squared_error: 0.1696
Epoch 252/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2066 - mean_squared_error: 0.1769 - val_loss: 0.2025 - val_mean_squared_error: 0.1729
Epoch 253/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2078 - mean_squared_error: 0.1781 - val_loss: 0.1985 - val_mean_squared_error: 0.1687
Epoch 254/2000
18/18 [==============================] - 1s 48ms/step - loss: 0.2085 - mean_squared_error: 0.1789 - val_loss: 0.2012 - val_mean_squared_error: 0.1714
Epoch 255/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2069 - mean_squared_error: 0.1773 - val_loss: 0.1954 - val_mean_squared_error: 0.1658
Epoch 256/2000
18/18 [==============================] - 1s 47ms/step - loss: 0.2070 - mean_squared_error: 0.1774 - val_loss: 0.1969 - val_mean_squared_error: 0.1675
Epoch 257/

Epoch 301/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.1948 - mean_squared_error: 0.1669 - val_loss: 0.2019 - val_mean_squared_error: 0.1739
Epoch 302/2000
18/18 [==============================] - 1s 48ms/step - loss: 0.1941 - mean_squared_error: 0.1664 - val_loss: 0.1976 - val_mean_squared_error: 0.1696
Epoch 303/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.1924 - mean_squared_error: 0.1646 - val_loss: 0.1983 - val_mean_squared_error: 0.1704
Epoch 304/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1919 - mean_squared_error: 0.1641 - val_loss: 0.2147 - val_mean_squared_error: 0.1869
Epoch 305/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.1913 - mean_squared_error: 0.1636 - val_loss: 0.2036 - val_mean_squared_error: 0.1759
Epoch 306/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1942 - mean_squared_error: 0.1665 - val_loss: 0.2047 - val_mean_squared_error: 0.1771
Epoch 307/

Epoch 351/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1577 - mean_squared_error: 0.1319 - val_loss: 0.2223 - val_mean_squared_error: 0.1965
Epoch 352/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1580 - mean_squared_error: 0.1323 - val_loss: 0.2123 - val_mean_squared_error: 0.1865
Epoch 353/2000
18/18 [==============================] - 1s 46ms/step - loss: 0.1591 - mean_squared_error: 0.1335 - val_loss: 0.2158 - val_mean_squared_error: 0.1901
Epoch 354/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1540 - mean_squared_error: 0.1283 - val_loss: 0.2025 - val_mean_squared_error: 0.1768
Epoch 355/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1530 - mean_squared_error: 0.1274 - val_loss: 0.2138 - val_mean_squared_error: 0.1882
Epoch 356/2000
18/18 [==============================] - 1s 45ms/step - loss: 0.1534 - mean_squared_error: 0.1279 - val_loss: 0.2017 - val_mean_squared_error: 0.1763
Epoch 357/